In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth

# Registration and creation of domain

### 1. Follow instructions to create [AAP test accout](https://submission-test.ebi.ac.uk/api/docs/guide_accounts_and_logging_in.html) or [AAP production account](https://submission.ebi.ac.uk/api/docs/guide_accounts_and_logging_in.html)

In [ ]:
# Assign your AAP username and accout to these variables
AAP_USERNAME = ""
AAP_PASSWORD = ""

In [ ]:
# Comment out these lines if you don't want to use test server for submission
aap_server = 'https://explore.api.aai.ebi.ac.uk'
submission_server = 'https://wwwdev.ebi.ac.uk'

# Uncomment these line if you want to use production server
# aap_server = 'https://api.aai.ebi.ac.uk'
# submission_server = 'https://www.ebi.ac.uk'

### 2. Use this function to get token (valid for 1 hour only)

In [ ]:
def get_token():
    response = requests.get(f"{aap_server}/auth", auth=HTTPBasicAuth(AAP_USERNAME, AAP_PASSWORD))
    return response.text

In [ ]:
TOKEN = get_token()

### 3*. Create a domain (optional)

In [ ]:
# Provide domain name
domain_name = ''
# Provide domain description
domain_desc = ''
domain_data = {
    'domainName': domain_name,
    'domainDesc': domain_desc
}
domain_data = json.dumps(domain_data)

In [ ]:
create_domain_response = requests.post(f"{aap_server}/domains", 
                                       headers={'Content-Type': 'application/json',
                                                'Accept': 'application/hal+json',
                                                'Authorization': f'Bearer {TOKEN}'}, 
                                       data=domain_data)

In [ ]:
TOKEN = get_token()

### 3. Choose domain

In [ ]:
choose_domain_response = requests.get(f"{aap_server}/my/domains", 
                                     headers={'Content-Type': 'application/json',
                                              'Accept': 'application/hal+json',
                                              'Authorization': f'Bearer {TOKEN}'})

In [ ]:
for domain in choose_domain_response.json():
    print(domain['domainName'])

In [ ]:
# from domain names above choose one and asign it to this variable
domain_name = ''

# Create submission

In [ ]:
# Provide path to samples.json file
samples_json_path = ''
with open(samples_json_path, 'r') as f:
    data = json.loads(f.read())

In [ ]:
# submit records without relationships part
biosamples_ids = dict()
with open('submission_results.txt', 'w') as f:
    for item in data:
        item['domain'] = domain_name
        del item['relationships']
        name = item['name']
        item = json.dumps(item)
        create_submission_response = requests.post(f"{submission_server}/biosamples/samples",
                                                   headers={'Content-Type': 'application/json',
                                                            'Accept': 'application/hal+json',
                                                            'Authorization': f'Bearer {TOKEN}'}, 
                                                   data=item)
        biosamples_ids[name] = create_submission_response.json()['accession']
        f.write(f"{name}\t{create_submission_response.json()['accession']}\n")

In [ ]:
# update record with relationships part
with open(samples_json_path, 'r') as f:
    data = json.loads(f.read())
for item in data:
    if len(item['relationships']) > 0:
        for relationship in item['relationships']:
            if relationship['source'] in biosamples_ids:
                relationship['source'] = biosamples_ids[relationship['source']]
            if relationship['target'] in biosamples_ids:
                relationship['target'] = biosamples_ids[relationship['target']]
        biosample_id = biosamples_ids[item['name']]
        item['accession'] = biosample_id
        item['domain'] = domain_name
        item = json.dumps(item)
        create_submission_response = requests.put(f"{submission_server}/biosamples/samples/{biosample_id}",
                                                   headers={'Content-Type': 'application/json',
                                                            'Accept': 'application/hal+json',
                                                            'Authorization': f'Bearer {TOKEN}'}, 
                                                   data=item)